# 🏛️ PKI (Public Key Infrastructure) 완전 정복

## 🎯 학습 목표
- PKI가 왜 필요한지 이해 (공개키 신뢰 문제)
- 인증서(Certificate)와 CA(Certificate Authority) 역할
- 신뢰 체인(Chain of Trust) 구조 완전 이해
- TLS/SSL이 어떻게 안전한 통신을 보장하는지 체험
- 실제 웹사이트 인증서 분석 실습

---


## 🤔 문제 상황: 공개키를 어떻게 믿을까?

### 💭 **시나리오: Alice와 Bob의 딜레마**

```
Alice: "이게 내 공개키야: 1A2B3C4D..."
Bob: "어떻게 이게 정말 Alice 것인지 확신할 수 있지? 🤷‍♂️"
```

### 🚨 **공개키의 신뢰 문제들:**

| 상황 | 문제점 | 위험도 |
|-----|-------|-------|
| 🎭 **신원 위조** | Eve가 Alice 행세 | 🔥🔥🔥 높음 |
| 📱 **키 교환** | 전화/메일로 공개키 전달 | 🔥🔥 중간 |
| 🌐 **웹사이트** | naver.com이 진짜인지 확인? | 🔥🔥🔥 높음 |
| 🏦 **은행 거래** | 은행 공개키 신뢰 근거? | 🔥🔥🔥🔥 매우 높음 |

### 💡 **해결책: 신뢰할 수 있는 제3자가 보증!**
- 🏛️ **정부**: "이 사람이 진짜 김철수입니다" (주민등록증)
- 🏦 **은행**: "이 계좌는 진짜 김철수 것입니다" (통장)
- 🔐 **CA (Certificate Authority)**: "이 공개키는 진짜 naver.com 것입니다" (인증서)

---


In [23]:
# 🔧 라이브러리 준비
import hashlib
import random
from datetime import datetime, timedelta

print("🏛️ PKI (Public Key Infrastructure) 마스터 과정!")
print("=" * 60)


🏛️ PKI (Public Key Infrastructure) 마스터 과정!


## 📜 인증서 (Digital Certificate) 이해

### 🆔 **디지털 인증서 = 디지털 신분증**

#### 📋 **현실 vs 디지털 신분증 비교:**

| 항목 | 현실 신분증 | 디지털 인증서 |
|-----|------------|-------------|
| **신원 정보** | 이름, 주소, 생년월일 | 도메인명, 회사명, 국가 |
| **사진** | 본인 얼굴 | 공개키 |
| **발급기관** | 정부 (행정안전부) | CA (VeriSign, Let's Encrypt) |
| **서명** | 공무원 도장 | CA의 디지털 서명 |
| **유효기간** | 10년 | 1~2년 |
| **위조 방지** | 특수 용지, 홀로그램 | 암호학적 해시, 서명 |

### 📝 **인증서 구조 (X.509 표준):**

```
┌─────────────────────────────────┐
│ 📋 인증서 (Certificate)          │
├─────────────────────────────────┤
│ 🆔 주체: naver.com               │
│ 🔑 공개키: 30 82 01 0A...        │
│ 🏛️ 발급자: GlobalSign CA         │
│ 📅 유효기간: 2024-01 ~ 2025-01    │
│ 🔐 용도: SSL/TLS, 이메일           │
├─────────────────────────────────┤
│ 🖋️ CA 서명: A5 B2 7F...          │
└─────────────────────────────────┘
```

### 🔍 **인증서 검증 과정:**
1. **인증서 받기**: 웹사이트에서 인증서 다운로드
2. **CA 확인**: 신뢰할 수 있는 CA가 발급했는지 확인
3. **서명 검증**: CA의 공개키로 서명 검증
4. **유효기간**: 만료되지 않았는지 확인
5. **취소 확인**: CRL(Certificate Revocation List) 체크

---


In [24]:
# 🏛️ 간단한 CA (Certificate Authority) 구현

class SimpleCertificate:
    """간단한 디지털 인증서 클래스"""
    def __init__(self, subject, public_key, issuer, valid_from, valid_to):
        self.subject = subject  # 인증서 주체 (예: naver.com)
        self.public_key = public_key  # 주체의 공개키
        self.issuer = issuer  # 발급자 (CA)
        self.valid_from = valid_from  # 유효 시작일
        self.valid_to = valid_to  # 유효 만료일
        self.signature = None  # CA의 서명 (나중에 추가)
    
    def to_string(self):
        """인증서를 문자열로 변환 (서명용)"""
        return f"{self.subject}|{self.public_key}|{self.issuer}|{self.valid_from}|{self.valid_to}"
    
    def is_valid(self, current_time=None):
        """인증서 유효성 검사"""
        if current_time is None:
            current_time = datetime.now()
        
        return self.valid_from <= current_time <= self.valid_to
    
    def display(self):
        """인증서 정보 출력"""
        print(f"\n📜 디지털 인증서")
        print(f"   🆔 주체(Subject): {self.subject}")
        print(f"   🔑 공개키: {self.public_key[:20]}...")
        print(f"   🏛️ 발급자(Issuer): {self.issuer}")
        print(f"   📅 유효기간: {self.valid_from.strftime('%Y-%m-%d')} ~ {self.valid_to.strftime('%Y-%m-%d')}")
        print(f"   ✅ 상태: {'유효' if self.is_valid() else '만료'}")
        if self.signature:
            print(f"   🖋️ CA 서명: {self.signature[:20]}...")

class SimpleCA:
    """간단한 인증기관 (Certificate Authority) 클래스"""
    def __init__(self, name):
        self.name = name
        # CA의 RSA 키 쌍 생성 (간단화)
        self.private_key = 12345  # CA의 개인키 (실제로는 매우 복잡)
        self.public_key = "CA_PUBLIC_KEY_67890"  # CA의 공개키
        
        print(f"🏛️ CA '{self.name}' 설립!")
        print(f"   🔑 CA 공개키: {self.public_key}")
    
    def issue_certificate(self, subject, subject_public_key, validity_days=365):
        """인증서 발급"""
        # 유효기간 설정
        now = datetime.now()
        valid_from = now
        valid_to = now + timedelta(days=validity_days)
        
        # 인증서 생성
        cert = SimpleCertificate(
            subject=subject,
            public_key=subject_public_key,
            issuer=self.name,
            valid_from=valid_from,
            valid_to=valid_to
        )
        
        # CA가 인증서에 서명
        cert.signature = self.sign_certificate(cert)
        
        print(f"\n🎫 인증서 발급 완료!")
        print(f"   📋 주체: {subject}")
        print(f"   🏛️ 발급자: {self.name}")
        print(f"   📅 유효기간: {validity_days}일")
        
        return cert
    
    def sign_certificate(self, certificate):
        """CA가 인증서에 디지털 서명"""
        # 인증서 내용을 해시화
        cert_data = certificate.to_string()
        cert_hash = hashlib.sha256(cert_data.encode()).hexdigest()
        
        # CA의 개인키로 서명 (실제로는 RSA/ECDSA)
        signature = hashlib.sha256((cert_hash + str(self.private_key)).encode()).hexdigest()
        
        return signature
    
    def verify_certificate(self, certificate):
        """인증서 검증"""
        # 재계산된 서명과 비교
        expected_signature = self.sign_certificate(certificate)
        
        signature_valid = (certificate.signature == expected_signature)
        time_valid = certificate.is_valid()
        
        print(f"\n🔍 인증서 검증 결과:")
        print(f"   🖋️ 서명 유효성: {'✅ 유효' if signature_valid else '❌ 무효'}")
        print(f"   ⏰ 시간 유효성: {'✅ 유효' if time_valid else '❌ 만료'}")
        print(f"   🎯 전체 결과: {'✅ 신뢰 가능' if (signature_valid and time_valid) else '❌ 신뢰 불가'}")
        
        return signature_valid and time_valid

# 🏛️ CA 설립 (실제 예: VeriSign, GlobalSign, Let's Encrypt)
global_ca = SimpleCA("GlobalTrust CA")


🏛️ CA 'GlobalTrust CA' 설립!
   🔑 CA 공개키: CA_PUBLIC_KEY_67890


In [25]:
# 🌐 웹사이트들이 인증서를 받는 과정

print("🌐 웹사이트들의 인증서 신청 과정")
print("=" * 50)

# 네이버가 CA에 인증서 신청
naver_public_key = "NAVER_PUBLIC_KEY_ABC123"
naver_cert = global_ca.issue_certificate(
    subject="naver.com",
    subject_public_key=naver_public_key,
    validity_days=365
)

# 구글이 CA에 인증서 신청
google_public_key = "GOOGLE_PUBLIC_KEY_XYZ789"
google_cert = global_ca.issue_certificate(
    subject="google.com",
    subject_public_key=google_public_key,
    validity_days=90  # 구글은 짧은 기간 선호
)

# 🚨 가짜 사이트가 네이버 행세 시도해보기!
print(f"\n🚨 해커의 가짜 네이버 사이트 시도...")
print("=" * 50)

# 해커가 자신만의 가짜 CA 만들기
fake_ca = SimpleCA("HackerCA (가짜)")

# 해커가 가짜 네이버 인증서 발급 시도
fake_naver_public_key = "FAKE_NAVER_KEY_HACKER123"
fake_naver_cert = fake_ca.issue_certificate(
    subject="naver.com",  # 같은 도메인 이름 사용!
    subject_public_key=fake_naver_public_key,
    validity_days=365
)

print(f"\n🎭 해커: '내가 만든 가짜 네이버 인증서다!'")
fake_naver_cert.display()

# 👀 사용자 관점: 인증서 확인하기
print("\n👤 사용자 Bob이 웹사이트 방문...")
print("=" * 40)

print(f"🌐 Bob: 'naver.com에 접속하려고 해'")

# 1. 진짜 네이버 인증서 확인
print(f"\n📜 상황 1: 진짜 naver.com 접속")
naver_cert.display()

print(f"\n🔍 Bob의 브라우저가 자동으로 인증서 검증...")
is_trusted_real = global_ca.verify_certificate(naver_cert)

if is_trusted_real:
    print(f"\n🎉 Bob: '안전한 사이트네! 로그인해도 되겠어' 🔒")
    print(f"   브라우저 주소창: 🔒 https://naver.com (안전함)")
else:
    print(f"\n⚠️ Bob: '뭔가 이상해... 접속하지 말자' 🚫")
    print(f"   브라우저 경고: ⚠️ 이 사이트는 안전하지 않습니다")

# 2. 가짜 네이버 인증서 확인
print(f"\n📜 상황 2: 가짜 naver.com 접속 (해커 사이트)")
fake_naver_cert.display()

print(f"\n🔍 Bob의 브라우저가 자동으로 인증서 검증...")
# 가짜 CA는 브라우저가 신뢰하지 않으므로 검증 실패!
is_trusted_fake = global_ca.verify_certificate(fake_naver_cert)

if is_trusted_fake:
    print(f"\n😱 이건 일어나면 안 되는 상황! 가짜가 통과됨!")
else:
    print(f"\n🛡️ Bob의 브라우저: '이 사이트는 안전하지 않습니다!'")
    print(f"   🔴 경고창: ⚠️ 인증서가 신뢰할 수 없는 기관에서 발급됨")
    print(f"   😰 Bob: '피싱 사이트구나! 절대 접속하지 말자'")

print(f"\n💡 PKI가 우리를 보호하는 방법:")
print(f"   ✅ 진짜 CA (GlobalTrust CA): 브라우저가 신뢰")
print(f"   ❌ 가짜 CA (HackerCA): 브라우저가 거부")
print(f"   🎯 결과: 가짜 사이트 자동 차단! 🛡️")


🌐 웹사이트들의 인증서 신청 과정

🎫 인증서 발급 완료!
   📋 주체: naver.com
   🏛️ 발급자: GlobalTrust CA
   📅 유효기간: 365일

🎫 인증서 발급 완료!
   📋 주체: google.com
   🏛️ 발급자: GlobalTrust CA
   📅 유효기간: 90일

🚨 해커의 가짜 네이버 사이트 시도...
🏛️ CA 'HackerCA (가짜)' 설립!
   🔑 CA 공개키: CA_PUBLIC_KEY_67890

🎫 인증서 발급 완료!
   📋 주체: naver.com
   🏛️ 발급자: HackerCA (가짜)
   📅 유효기간: 365일

🎭 해커: '내가 만든 가짜 네이버 인증서다!'

📜 디지털 인증서
   🆔 주체(Subject): naver.com
   🔑 공개키: FAKE_NAVER_KEY_HACKE...
   🏛️ 발급자(Issuer): HackerCA (가짜)
   📅 유효기간: 2025-09-17 ~ 2026-09-17
   ✅ 상태: 유효
   🖋️ CA 서명: 4dc9e706a701ca59d987...

👤 사용자 Bob이 웹사이트 방문...
🌐 Bob: 'naver.com에 접속하려고 해'

📜 상황 1: 진짜 naver.com 접속

📜 디지털 인증서
   🆔 주체(Subject): naver.com
   🔑 공개키: NAVER_PUBLIC_KEY_ABC...
   🏛️ 발급자(Issuer): GlobalTrust CA
   📅 유효기간: 2025-09-17 ~ 2026-09-17
   ✅ 상태: 유효
   🖋️ CA 서명: 5684c2d192458ad71472...

🔍 Bob의 브라우저가 자동으로 인증서 검증...

🔍 인증서 검증 결과:
   🖋️ 서명 유효성: ✅ 유효
   ⏰ 시간 유효성: ✅ 유효
   🎯 전체 결과: ✅ 신뢰 가능

🎉 Bob: '안전한 사이트네! 로그인해도 되겠어' 🔒
   브라우저 주소창: 🔒 https://naver.com (안전함)

📜 상황 2: 가짜 naver.

## 🔗 신뢰 체인 (Chain of Trust)

### 🏗️ **신뢰의 계층 구조**

#### 🎯 **문제**: CA를 누가 보증하지?
- 🤔 "GlobalTrust CA를 어떻게 믿을 수 있을까?"
- 🔄 "CA의 CA가 필요하다!"

#### 🏛️ **해결책**: 신뢰 체인 (Certificate Chain)

```
🏔️ Root CA (최상위)
   └── "우리가 스스로를 보증한다" (Self-Signed)
   └── 브라우저/OS에 미리 설치
       ↓ 서명
🏛️ Intermediate CA (중간)
   └── "Root CA가 우리를 보증한다"
   └── 실제 인증서 발급 업무
       ↓ 서명
📜 End Entity Certificate (최종)
   └── "Intermediate CA가 우리를 보증한다"
   └── 실제 웹사이트 (naver.com)
```

### 🔍 **신뢰 체인 검증 과정:**

1. **웹사이트 인증서** 받기
2. **Intermediate CA 인증서** 받기
3. **Root CA 인증서** 찾기 (브라우저에 내장)
4. **역순으로 검증**: Root → Intermediate → Website
5. **모든 단계 통과** → ✅ 신뢰 가능


In [26]:
# 🔗 신뢰 체인 시뮬레이션

class TrustChain:
    """신뢰 체인 관리 클래스"""
    def __init__(self):
        self.root_cas = {}  # 신뢰할 수 있는 Root CA들
        print("🏔️ 신뢰 체인 시스템 초기화")
    
    def add_trusted_root(self, ca_name, ca_public_key):
        """신뢰할 수 있는 Root CA 추가 (브라우저/OS가 하는 일)"""
        self.root_cas[ca_name] = ca_public_key
        print(f"🏛️ Root CA 추가: {ca_name}")
    
    def verify_chain(self, certificates):
        """인증서 체인 검증"""
        print(f"\n🔍 신뢰 체인 검증 시작...")
        print(f"   📋 체인 길이: {len(certificates)}개")
        
        # 가장 상위 인증서가 신뢰할 수 있는 Root CA인지 확인
        root_cert = certificates[-1]  # 마지막이 Root
        
        if root_cert.issuer in self.root_cas:
            print(f"   ✅ Root CA '{root_cert.issuer}' 신뢰 가능")
            
            # 체인의 각 링크 검증
            for i in range(len(certificates) - 1):
                current_cert = certificates[i]
                issuer_cert = certificates[i + 1]
                
                print(f"   🔗 {current_cert.subject} ← {issuer_cert.issuer}")
                
                # 실제로는 디지털 서명 검증
                if current_cert.issuer == issuer_cert.subject:
                    print(f"     ✅ 서명 검증 성공")
                else:
                    print(f"     ❌ 서명 검증 실패")
                    return False
            
            print(f"\n🎉 전체 신뢰 체인 검증 성공!")
            return True
        else:
            print(f"   ❌ Root CA '{root_cert.issuer}' 신뢰할 수 없음")
            return False

# 🌐 브라우저/OS 시뮬레이션 (신뢰할 수 있는 Root CA들 설정)
browser = TrustChain()
browser.add_trusted_root("VeriSign Root CA", "VERISIGN_ROOT_KEY")
browser.add_trusted_root("GlobalSign Root CA", "GLOBALSIGN_ROOT_KEY")
browser.add_trusted_root("Let's Encrypt Root CA", "LETSENCRYPT_ROOT_KEY")

print(f"\n📱 브라우저가 신뢰하는 Root CA: {len(browser.root_cas)}개")


🏔️ 신뢰 체인 시스템 초기화
🏛️ Root CA 추가: VeriSign Root CA
🏛️ Root CA 추가: GlobalSign Root CA
🏛️ Root CA 추가: Let's Encrypt Root CA

📱 브라우저가 신뢰하는 Root CA: 3개


In [27]:
# 🏗️ 현실적인 신뢰 체인 구축

print("🏗️ 현실적인 PKI 체인 구축")
print("=" * 40)

# 1. Root CA (브라우저에 내장)
root_ca = SimpleCA("VeriSign Root CA")

# 2. Intermediate CA (실제 업무용)
intermediate_ca = SimpleCA("VeriSign Class 3 CA")

# 3. Root CA가 Intermediate CA 인증서 발급
intermediate_cert = root_ca.issue_certificate(
    subject="VeriSign Class 3 CA",
    subject_public_key=intermediate_ca.public_key,
    validity_days=3650  # 10년 (Root는 오래)
)

# 4. Intermediate CA가 웹사이트 인증서 발급
bank_public_key = "BANK_PUBLIC_KEY_SECURE123"
bank_cert = intermediate_ca.issue_certificate(
    subject="kookmin-bank.com",
    subject_public_key=bank_public_key,
    validity_days=365  # 1년 (일반적)
)

# 5. 브라우저에 Root CA 추가
browser.add_trusted_root("VeriSign Root CA", root_ca.public_key)

print(f"\n🏛️ PKI 체인 구축 완료!")
print(f"   🏔️ Root: VeriSign Root CA")
print(f"   🏛️ Intermediate: VeriSign Class 3 CA")
print(f"   🏦 End Entity: kookmin-bank.com")


🏗️ 현실적인 PKI 체인 구축
🏛️ CA 'VeriSign Root CA' 설립!
   🔑 CA 공개키: CA_PUBLIC_KEY_67890
🏛️ CA 'VeriSign Class 3 CA' 설립!
   🔑 CA 공개키: CA_PUBLIC_KEY_67890

🎫 인증서 발급 완료!
   📋 주체: VeriSign Class 3 CA
   🏛️ 발급자: VeriSign Root CA
   📅 유효기간: 3650일

🎫 인증서 발급 완료!
   📋 주체: kookmin-bank.com
   🏛️ 발급자: VeriSign Class 3 CA
   📅 유효기간: 365일
🏛️ Root CA 추가: VeriSign Root CA

🏛️ PKI 체인 구축 완료!
   🏔️ Root: VeriSign Root CA
   🏛️ Intermediate: VeriSign Class 3 CA
   🏦 End Entity: kookmin-bank.com


In [28]:
# 🔍 사용자가 은행 사이트 접속 시 체인 검증

print("\n💳 사용자가 은행 사이트 접속...")
print("=" * 50)

# 브라우저가 받는 인증서 체인
certificate_chain = [
    bank_cert,         # 1. 은행 인증서
    intermediate_cert, # 2. Intermediate CA 인증서
    # Root CA 인증서는 브라우저에 내장되어 있음
]

print(f"📥 브라우저가 받은 인증서 체인:")
for i, cert in enumerate(certificate_chain, 1):
    print(f"   {i}. {cert.subject} (발급자: {cert.issuer})")

# 체인 검증 수행
is_chain_valid = browser.verify_chain(certificate_chain + [
    SimpleCertificate(
        subject="VeriSign Root CA",
        public_key=root_ca.public_key,
        issuer="VeriSign Root CA",  # Self-Signed
        valid_from=datetime.now(),
        valid_to=datetime.now() + timedelta(days=7300)  # 20년
    )
])

if is_chain_valid:
    print(f"\n🔒 브라우저: 'kookmin-bank.com은 안전합니다'")
    print(f"   💚 주소창: 🔒 https://kookmin-bank.com")
    print(f"   💳 사용자: '안심하고 인터넷뱅킹 하자!'")
else:
    print(f"\n⚠️ 브라우저: '이 사이트는 신뢰할 수 없습니다'")
    print(f"   🔴 주소창: ⚠️ 연결이 보안되지 않음")
    print(f"   😰 사용자: '다른 방법을 찾아야겠어...'")

print(f"\n🎯 PKI 체인 검증 완료! 신뢰할 수 있는 통신 경로 확보!")



💳 사용자가 은행 사이트 접속...
📥 브라우저가 받은 인증서 체인:
   1. kookmin-bank.com (발급자: VeriSign Class 3 CA)
   2. VeriSign Class 3 CA (발급자: VeriSign Root CA)

🔍 신뢰 체인 검증 시작...
   📋 체인 길이: 3개
   ✅ Root CA 'VeriSign Root CA' 신뢰 가능
   🔗 kookmin-bank.com ← VeriSign Root CA
     ✅ 서명 검증 성공
   🔗 VeriSign Class 3 CA ← VeriSign Root CA
     ✅ 서명 검증 성공

🎉 전체 신뢰 체인 검증 성공!

🔒 브라우저: 'kookmin-bank.com은 안전합니다'
   💚 주소창: 🔒 https://kookmin-bank.com
   💳 사용자: '안심하고 인터넷뱅킹 하자!'

🎯 PKI 체인 검증 완료! 신뢰할 수 있는 통신 경로 확보!


## 🌐 TLS/SSL: PKI의 실전 활용

### 🔒 **HTTPS = HTTP + TLS/SSL**

#### 📝 **TLS Handshake 과정 (간단화):**

```
👤 Client                     🖥️ Server
   │                             │
   ├─ 1. "안녕, HTTPS 지원해?" ──→│
   │                             │
   │←── 2. "응! 내 인증서야" ─────┤
   │     📜 [인증서 + 공개키]      │
   │                             │
   ├─ 3. 🔍 인증서 검증           │
   │     (PKI 체인 확인)           │
   │                             │
   ├─ 4. 🔑 대칭키 생성 ──────────→│
   │     (서버 공개키로 암호화)     │
   │                             │
   │←─── 5. "대칭키 받았어!" ─────┤
   │                             │
   ├─ 6. 🔒 암호화 통신 시작 ─────→│
   │     (대칭키로 빠른 암호화)     │
```

### 🎯 **TLS의 핵심 아이디어:**
1. **PKI로 신원 확인** ("정말 네이버 맞아?")
2. **공개키로 대칭키 교환** ("우리만의 비밀 열쇠 만들자")
3. **대칭키로 빠른 통신** ("이제 빠르게 대화하자")

### 💡 **왜 이렇게 복잡하게?**
- 🏃‍♂️ **대칭 암호**: 빠르지만 키 교환 문제
- 🐌 **공개키 암호**: 안전하지만 느림
- 🧠 **TLS 해결책**: 둘의 장점만 결합!

---


In [29]:
# 🌐 TLS Handshake 시뮬레이션

class TLSClient:
    """TLS 클라이언트 (브라우저) 시뮬레이션"""
    def __init__(self, name):
        self.name = name
        self.trust_chain = TrustChain()
        # 브라우저의 신뢰할 수 있는 Root CA들
        self.trust_chain.add_trusted_root("VeriSign Root CA", "VERISIGN_ROOT")
        self.symmetric_key = None
        
        print(f"👤 TLS 클라이언트 '{self.name}' 준비 완료")
    
    def connect_to_server(self, server):
        """서버에 TLS 연결 시도"""
        print(f"\n🔗 {self.name} → {server.domain} 연결 시도...")
        print(f"=" * 50)
        
        # 1. Client Hello
        print(f"1️⃣ {self.name}: '안녕, TLS 연결하고 싶어!'")
        
        # 2. Server Hello + Certificate
        server_cert = server.send_certificate()
        print(f"2️⃣ {server.domain}: '내 인증서야!'")
        server_cert.display()
        
        # 3. Certificate Verification
        print(f"\n3️⃣ {self.name}: '인증서 검증 중...'")
        if self.verify_server_certificate(server_cert):
            print(f"   ✅ 인증서 유효! 신뢰할 수 있는 서버")
            
            # 4. Key Exchange
            self.symmetric_key = f"SECRET_KEY_{random.randint(1000, 9999)}"
            encrypted_key = server.encrypt_with_public_key(self.symmetric_key)
            
            print(f"\n4️⃣ {self.name}: '대칭키 생성하고 공개키로 암호화해서 전송!'")
            print(f"   🔑 대칭키: {self.symmetric_key}")
            print(f"   🔒 암호화된 키: {encrypted_key[:20]}...")
            
            # 5. Server confirms
            server.receive_symmetric_key(encrypted_key)
            
            # 6. Secure communication
            print(f"\n6️⃣ 🎉 TLS 연결 성공!")
            print(f"   🔒 이제 모든 통신이 {self.symmetric_key}로 암호화됨")
            print(f"   💚 브라우저 주소창: 🔒 https://{server.domain}")
            
            return True
        else:
            print(f"   ❌ 인증서 무효! 연결 거부")
            print(f"   🔴 브라우저 경고: '이 사이트는 안전하지 않습니다'")
            return False
    
    def verify_server_certificate(self, certificate):
        """서버 인증서 검증 (PKI 체인)"""
        # 실제로는 복잡한 체인 검증
        return certificate.is_valid() and certificate.issuer in ["VeriSign Root CA", "GlobalTrust CA"]

class TLSServer:
    """TLS 서버 시뮬레이션"""
    def __init__(self, domain, certificate):
        self.domain = domain
        self.certificate = certificate
        self.private_key = f"{domain}_private_key"
        self.symmetric_key = None
        
        print(f"🖥️ TLS 서버 '{self.domain}' 시작")
    
    def send_certificate(self):
        """클라이언트에게 인증서 전송"""
        return self.certificate
    
    def encrypt_with_public_key(self, data):
        """공개키로 데이터 암호화 (실제로는 RSA/ECC)"""
        return hashlib.sha256((data + self.certificate.public_key).encode()).hexdigest()
    
    def receive_symmetric_key(self, encrypted_key):
        """암호화된 대칭키 받고 복호화"""
        # 실제로는 개인키로 복호화
        print(f"5️⃣ {self.domain}: '대칭키 받았어! 개인키로 복호화 성공!'")
        self.symmetric_key = "DECRYPTED_SYMMETRIC_KEY"

# 🖥️ 서버 설정 (예: 네이버)
naver_server = TLSServer("naver.com", naver_cert)

# 👤 클라이언트 설정 (사용자)
user_browser = TLSClient("Chrome 브라우저")

# 🔗 TLS 연결 시도
connection_success = user_browser.connect_to_server(naver_server)


🖥️ TLS 서버 'naver.com' 시작
🏔️ 신뢰 체인 시스템 초기화
🏛️ Root CA 추가: VeriSign Root CA
👤 TLS 클라이언트 'Chrome 브라우저' 준비 완료

🔗 Chrome 브라우저 → naver.com 연결 시도...
1️⃣ Chrome 브라우저: '안녕, TLS 연결하고 싶어!'
2️⃣ naver.com: '내 인증서야!'

📜 디지털 인증서
   🆔 주체(Subject): naver.com
   🔑 공개키: NAVER_PUBLIC_KEY_ABC...
   🏛️ 발급자(Issuer): GlobalTrust CA
   📅 유효기간: 2025-09-17 ~ 2026-09-17
   ✅ 상태: 유효
   🖋️ CA 서명: 5684c2d192458ad71472...

3️⃣ Chrome 브라우저: '인증서 검증 중...'
   ✅ 인증서 유효! 신뢰할 수 있는 서버

4️⃣ Chrome 브라우저: '대칭키 생성하고 공개키로 암호화해서 전송!'
   🔑 대칭키: SECRET_KEY_6609
   🔒 암호화된 키: 2bdfec4d089d7c2785f7...
5️⃣ naver.com: '대칭키 받았어! 개인키로 복호화 성공!'

6️⃣ 🎉 TLS 연결 성공!
   🔒 이제 모든 통신이 SECRET_KEY_6609로 암호화됨
   💚 브라우저 주소창: 🔒 https://naver.com


## 🎯 PKI의 실제 활용 사례

### 🌍 **어디서나 볼 수 있는 PKI:**

| 분야 | 사용 사례 | PKI 역할 |
|-----|----------|----------|
| 🌐 **웹 브라우징** | HTTPS, 온라인 쇼핑 | 웹사이트 신원 확인 |
| 📧 **이메일** | S/MIME, PGP | 발신자 인증, 암호화 |
| 💳 **금융** | 인터넷뱅킹, 카드결제 | 거래 보안, 부인방지 |
| 🏛️ **정부** | 전자서명, 공인인증서 | 법적 효력, 신원확인 |
| 📱 **모바일** | 앱 서명, HTTPS API | 앱 무결성, API 보안 |
| 🏢 **기업** | VPN, 내부 시스템 | 직원 인증, 기밀보호 |
| 🔐 **IoT** | 디바이스 인증 | 기기 신원, 펌웨어 검증 |

### 🏆 **주요 CA 업체들:**

#### 💰 **상용 CA:**
- **DigiCert**: 세계 1위, 고급 보안
- **GlobalSign**: 유럽 강세
- **Comodo/Sectigo**: 가성비 좋음

#### 🆓 **무료 CA:**
- **Let's Encrypt**: 무료 자동화, 90일 갱신
- **ZeroSSL**: Let's Encrypt 대안

#### 🏛️ **정부 CA:**
- **한국**: KISA, 금융결제원
- **미국**: Federal Bridge CA
- **중국**: CFCA

### 🔮 **PKI의 미래:**
- **Post-Quantum Cryptography**: 양자 컴퓨터 대비
- **Certificate Transparency**: 인증서 투명성
- **ACME Protocol**: 자동 인증서 관리
- **Blockchain + PKI**: 탈중앙화 신뢰

---


In [30]:
# 🎓 PKI 최종 정리 및 실습 완료

print("🏆 PKI (Public Key Infrastructure) 완전 정복!")
print("=" * 60)

print("\n📚 학습 완료 사항:")
print("\n🤔 문제 인식:")
print("   ✅ '공개키를 어떻게 믿을 수 있을까?' 문제 이해")
print("   ✅ 신원 위조, 중간자 공격의 위험성 체험")

print("\n📜 인증서 시스템:")
print("   ✅ 디지털 인증서 = 디지털 신분증 개념")
print("   ✅ CA (Certificate Authority) 역할과 책임")
print("   ✅ 인증서 발급, 검증, 폐기 과정")

print("\n🔗 신뢰 체인:")
print("   ✅ Root CA → Intermediate CA → End Entity 구조")
print("   ✅ 브라우저/OS의 Root CA 관리 방식")
print("   ✅ 체인 검증 과정과 보안 원리")

print("\n🌐 TLS/SSL 실전:")
print("   ✅ HTTPS = HTTP + TLS 이해")
print("   ✅ TLS Handshake 과정 체험")
print("   ✅ PKI + 대칭암호 하이브리드 방식")

print("\n🎯 실무 활용:")
print("   ✅ 웹 브라우징부터 IoT까지 PKI 적용 분야")
print("   ✅ 상용/무료/정부 CA 생태계 이해")
print("   ✅ 미래 기술 동향 (Post-Quantum, Blockchain)")

print("\n🚀 다음 단계:")
print("   🔮 Zero-Knowledge Proof (ZKP) 입문")
print("   🧙‍♂️ 알리바바 동굴로 ZKP 원리 체험")
print("   🏗️ zk-SNARKs vs zk-STARKs 비교")

print("\n✨ 축하합니다! PKI의 모든 것을 마스터하셨습니다! ✨")
print("\n💡 이제 웹에서 자물쇠 🔒 아이콘을 볼 때마다")
print("   복잡한 PKI 시스템이 뒤에서 작동하는 걸 이해하시겠네요! 😊")
print("\n🎯 암호학의 실용적 측면을 완전히 체득하셨어요! 🚀")


🏆 PKI (Public Key Infrastructure) 완전 정복!

📚 학습 완료 사항:

🤔 문제 인식:
   ✅ '공개키를 어떻게 믿을 수 있을까?' 문제 이해
   ✅ 신원 위조, 중간자 공격의 위험성 체험

📜 인증서 시스템:
   ✅ 디지털 인증서 = 디지털 신분증 개념
   ✅ CA (Certificate Authority) 역할과 책임
   ✅ 인증서 발급, 검증, 폐기 과정

🔗 신뢰 체인:
   ✅ Root CA → Intermediate CA → End Entity 구조
   ✅ 브라우저/OS의 Root CA 관리 방식
   ✅ 체인 검증 과정과 보안 원리

🌐 TLS/SSL 실전:
   ✅ HTTPS = HTTP + TLS 이해
   ✅ TLS Handshake 과정 체험
   ✅ PKI + 대칭암호 하이브리드 방식

🎯 실무 활용:
   ✅ 웹 브라우징부터 IoT까지 PKI 적용 분야
   ✅ 상용/무료/정부 CA 생태계 이해
   ✅ 미래 기술 동향 (Post-Quantum, Blockchain)

🚀 다음 단계:
   🔮 Zero-Knowledge Proof (ZKP) 입문
   🧙‍♂️ 알리바바 동굴로 ZKP 원리 체험
   🏗️ zk-SNARKs vs zk-STARKs 비교

✨ 축하합니다! PKI의 모든 것을 마스터하셨습니다! ✨

💡 이제 웹에서 자물쇠 🔒 아이콘을 볼 때마다
   복잡한 PKI 시스템이 뒤에서 작동하는 걸 이해하시겠네요! 😊

🎯 암호학의 실용적 측면을 완전히 체득하셨어요! 🚀
